<a href="https://colab.research.google.com/github/Shrzzzz/Global-Fragrance-Trends-Analysis/blob/main/Global_Scent_Trends_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import files
uploaded = files.upload()

Saving fra_cleaned.csv to fra_cleaned.csv


In [2]:
import pandas as pd

# Use the correct separator (;) and encoding
df = pd.read_csv("fra_cleaned.csv", sep=';', encoding='cp1252')

# Clean up column names
df.columns = df.columns.str.strip().str.lower()
print(df.columns.tolist())  # Optional: just to confirm


['url', 'perfume', 'brand', 'country', 'gender', 'rating value', 'rating count', 'year', 'top', 'middle', 'base', 'perfumer1', 'perfumer2', 'mainaccord1', 'mainaccord2', 'mainaccord3', 'mainaccord4', 'mainaccord5']


In [3]:
# Combine all note columns
df['all_notes'] = df[['top', 'middle', 'base']].fillna('').agg(','.join, axis=1)

# Split into list and explode
df['all_notes'] = df['all_notes'].str.split(',')
df = df.explode('all_notes')

# Clean note names
df['all_notes'] = df['all_notes'].str.strip().str.lower()


**Top fragrance notes by country**

In [4]:
import plotly.express as px

# Count of each note per country
note_counts = df.groupby(['country', 'all_notes']).size().reset_index(name='count')

# Top 5 notes per country
top_notes = note_counts.groupby('country').apply(lambda x: x.nlargest(5, 'count')).reset_index(drop=True)

# Plot for top 3 countries
top_countries = top_notes['country'].value_counts().head(3).index.tolist()
plot_data = top_notes[top_notes['country'].isin(top_countries)]

fig = px.bar(plot_data,
             x='all_notes',
             y='count',
             color='country',
             barmode='group',
             title='Top Fragrance Notes by Country')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

<ipython-input-4-d966e4f48996>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_notes = note_counts.groupby('country').apply(lambda x: x.nlargest(5, 'count')).reset_index(drop=True)


Perfume Launch trends by country

In [5]:

df['year'] = pd.to_numeric(df['year'], errors='coerce')

# Count perfumes launched per year per country
trend_data = df.dropna(subset=['year']).groupby(['year', 'country']).size().reset_index(name='perfume_count')

# Only keep top countries
top_trend_countries = trend_data['country'].value_counts().head(3).index.tolist()
trend_data = trend_data[trend_data['country'].isin(top_trend_countries)]

# Plot trend line
fig = px.line(trend_data,
              x='year',
              y='perfume_count',
              color='country',
              title='Perfume Launch Trends by Country Over Time',
              markers=True)
fig.show()


Top fragrance notes by gender

In [6]:
# Count note occurrences by gender
note_gender = df.groupby(['gender', 'all_notes']).size().reset_index(name='count')

# Top 5 notes per gender
top_gender_notes = note_gender.groupby('gender').apply(lambda x: x.nlargest(5, 'count')).reset_index(drop=True)

fig = px.bar(top_gender_notes,
             x='all_notes',
             y='count',
             color='gender',
             barmode='group',
             title='Top Fragrance Notes by Gender')
fig.update_layout(xaxis_tickangle=-45)
fig.show()


<ipython-input-6-d7a619bdae48>:5: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

